In [5]:
import os
import os.path as path
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.models import load_model

In [6]:
current_dir = os.path.join(os.getcwd())
file = os.path.join(path.dirname(path.dirname(current_dir)), "generate_data\data_gpa.csv")
myData = pd.read_csv(file, delimiter=',', usecols=['gpa','credit','time','difficulty', 'points'])
my_data_copy = myData
myData.shape

(4106, 5)

In [7]:
myData["gpa"] = myData["gpa"].values / 4
myData["credit"] = myData["credit"].values / 6
myData["time"] = myData["credit"].values / 10
myData ["difficulty"] = myData['difficulty'].values / 5
myData["points"] = myData["points"].values / 4